# Reconstructing the path

In this assignment you will use Spark to compute the shortest path between two vertices. In the video, you have learned how to compute the distances between a source vertex and all other vertices in a graph. Now, your task is to reconstruct the shortest path, that is a sequence of vertices connected by the edges.

Dataset location: */data/twitter/twitter_sample_small.txt*

Format: `user_id \t follower_id`

Your task is to find the shortest path between vertices 12 and 34. In case of multiple shortest paths (that is, disjoint paths with the same length), any will suffice. Output format is the sequence of vertices, delimited by a comma, without spaces. For example, the path `12 -> 42 -> 34` should be printed as:

```
12,42,34
```

The result on the sample dataset:

```
12,422,53,52,107,20,23,274,34
```

### Step 1. Create SparkContext.

In [ ]:
from pyspark import SparkConf, SparkContext

sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("yarn"))

# For local run uncomment the lines below.
# sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))
# sc.uiWebUrl

### Step 2. BFS

In [ ]:
# For remote run
DATA_FILE = "/data/twitter/twitter_sample_small.txt"

# For local run
# DATA_FILE = "twitter_sample_small.txt"

In [ ]:
def parse_edge(s):
    """
    Parses an edge to the format (<follower>, (<user>, <parent>)).

    Note, that <parent> is always -1.
    """
    user, follower = s.split("\t")
    return (int(follower), (int(user), -1))


def prepare_step(item):
    """
    Parses an edge possible way (edge + parent + neighbour).
    """
    follower, distance, neighbour = item[0], item[1][0][0], item[1][1][0]
    return (neighbour, (distance, follower))


def make_step(item):
    """
    Recalculates existed distance for a step and complete it.
    """
    user, old_step, new_step = item[0], item[1][0], item[1][1]
    if old_step:
        return (user, old_step)
    else:
        return (user, (new_step[0] + 1, new_step[1]))


def bfs(start, num_partitions=4):
    """
    Performs a breadth-first search across the graph from specified vertex.

    :return: a list of traversed vertexes in format: 
             [(<vertex>, (<distance>, <parent>))]
    """
    edges = sc.textFile(DATA_FILE).map(parse_edge).cache()

    current_distance = 0
    distances = sc.parallelize([(start, (current_distance, -1))]).partitionBy(num_partitions)

    while True:
        candidates = distances.join(edges, num_partitions).map(prepare_step)
        new_distances = distances.fullOuterJoin(candidates, num_partitions).map(make_step).distinct().persist()

        updated_distances_count = new_distances.filter(lambda i: i[1][0] == current_distance + 1).count()
        if updated_distances_count > 0:
            current_distance += 1
            distances = new_distances
        else:
            break
    
    return distances


### Step 3. Reconstruct the path.

In [ ]:
def reconstruct_path(traverse_table, start, end):
    """
    Builds a path from the start to the end from
    the traverse table which has the format:
    [(<vertex>, (<distance>, <parent>))] .

    :return: a path as a comma-separated string
    """
    path = []
    parent = end

    while parent != start and parent != -1:
        current_step = traverse_table.filter(lambda x: x[0] == parent).collect()
        path.append(parent)
        parent = int(current_step[0][1][1])

    if parent == -1:
        return ""

    path.append(start)
    return ",".join(list(map(str, path[::-1])))

### Step 4. Find the shortest path.

In [ ]:
start = 12
end = 34

traverse_table = bfs(start).cache()
print(reconstruct_path(traverse_table, start, end))

### Step 5. Free SparkContext.

In [ ]:
sc.stop()